# Εργασία Ανάκτησης Πληροφορίας
Δημιουργία μηχανής αναζήτησης

## Εισαγωγή βιβλιοθηκών

In [10]:
! pip install kaggle
! pip install rank-bm25

In [11]:
import os
import nltk
import re
import json

import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

from rank_bm25 import BM25Okapi

from collections import defaultdict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Βήμα 1. Συλλογή δεδομένων

In [4]:
# Εισαγωγή του API Key και του ονόματος χρήστη από το Kaggle (μέσω kaggle.json)
os.environ["KAGGLE_KEY"] = 'f8d5210b1add9082c88d61aed904ccab'
os.environ["KAGGLE_USERNAME"] = 'iaswnas'

# Λήψη του dataset από το Kaggle
!kaggle datasets download jrobischon/wikipedia-movie-plots

# Αποσυμπίεση του dataset
!unzip /content/wikipedia-movie-plots.zip

Dataset URL: https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots
License(s): CC-BY-SA-4.0
100% 29.9M/29.9M [00:02<00:00, 21.0MB/s]
100% 29.9M/29.9M [00:02<00:00, 12.5MB/s]
Archive:  /content/wikipedia-movie-plots.zip
  inflating: wiki_movie_plots_deduped.csv  


In [12]:
def preprocess_movie_plots(csv_filepath, output_filepath="preprocessed_plots.csv"):
    """Reads the original movie CSV, prepends titles to plots, saves combined text to a new CSV file."""

    try:
        df = pd.read_csv(csv_filepath)
    except FileNotFoundError:
        print(f"Error: File not found at {csv_filepath}")
        return

    if not all(col in df.columns for col in ["Title", "Plot"]):
        print("Error: CSV must have 'Title' and 'Plot' columns.")
        return

    df["Combined"] = df["Title"] + ": " + df["Plot"]

    # Keep only the "Combined" column
    df = df[["Combined"]]

    try:
        df.to_csv(output_filepath, index=False, header=False, encoding="utf-8")
        print(f"Preprocessed data saved to {output_filepath}")

    except Exception as e:
        print(f"Error saving CSV: {e}")

csv_filepath = "/content/wiki_movie_plots_deduped.csv"
preprocess_movie_plots(csv_filepath)

Preprocessed data saved to preprocessed_plots.csv


In [13]:
def reduce_csv_size(input_filepath, output_filepath, fraction=0.01):
    """Reduces the size of a CSV file by keeping a specified fraction of the rows."""
    try:
        df = pd.read_csv(input_filepath)
    except FileNotFoundError:
        print(f"Error: File not found at {input_filepath}")
        return

    reduced_df = df.sample(frac=fraction, random_state=42)  # a specific random state for reproducibility of results


    reduced_df.to_csv(output_filepath, index=False)
    print(f"Reduced CSV saved to {output_filepath}")

# Example usage in Colab:
input_csv = "/content/preprocessed_plots.csv"  # preprocessed CSV location
output_csv = "/content/reduced_movie_plots.csv" # New truncated CSV location
reduce_csv_size(input_csv, output_csv, fraction=0.01) # Keep a fraction of the original data

Reduced CSV saved to /content/reduced_movie_plots.csv


## Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):

In [14]:
def preprocess_text(text):
    """Preprocesses a single text string."""

    # 1. Lowercasing
    text = text.lower()

    # 2. Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces

    # 3. Tokenization
    tokens = nltk.word_tokenize(text)

    # 4. Stop word removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Stemming (or Lemmatization - uncomment the relevant part)
    stemmer = PorterStemmer()
    # lemmatizer = WordNetLemmatizer()  # lemmatization
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    # lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens] # lemmatization

    # 6. Join tokens back into a string
    processed_text = " ".join(stemmed_tokens)  # Or lemmatized_tokens

    return processed_text

def preprocess_csv(csv_filepath, output_filepath="preprocessed_data.csv"):
    """Preprocesses the text data in a CSV file."""

    try:
        df = pd.read_csv(csv_filepath, header=None, names=["text"])  # Assumes no header
    except FileNotFoundError:
        print(f"Error: File not found at {csv_filepath}")
        return


    df['processed_text'] = df['text'].apply(preprocess_text)

    df.to_csv(output_filepath, index=False, encoding="utf-8")  # Save to new CSV
    print(f"Preprocessed data saved to {output_filepath}")

# Pre-process dataset:
csv_filepath = "/content/reduced_movie_plots.csv"
preprocess_csv(csv_filepath)

Preprocessed data saved to preprocessed_data.csv


## Βήμα 3. Ευρετήριο (Indexing):

In [15]:
def create_inverted_index(csv_filepath, output_filepath="inverted_index.json"):
    """Creates an inverted index from the preprocessed CSV."""

    try:
        df = pd.read_csv(csv_filepath, header=None, names=["text"])
    except FileNotFoundError:
        print(f"Error: File not found at {csv_filepath}")
        return

    inverted_index = {}
    for doc_id, row in df.iterrows():  # Use doc_id as document identifier
        text = row['text']
        terms = text.split()  # Split terms

        for term in terms:
            if term not in inverted_index:
                inverted_index[term] = []  # Initialize postings list for the term
            inverted_index[term].append(doc_id)


    # Save the inverted index to a JSON file:
    with open(output_filepath, 'w', encoding='utf-8') as f:
        json.dump(inverted_index, f, indent=4)

    print(f"Inverted index saved to {output_filepath}")

# Finally, create the inverted index:
csv_filepath = "/content/reduced_movie_plots.csv"
create_inverted_index(csv_filepath)

Inverted index saved to inverted_index.json


## Βήμα 4. Μηχανή αναζήτησης (Search Engine)

In [17]:
def load_inverted_index(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_documents(filepath):
    df = pd.read_csv(filepath, header=None, names=["text"])
    return df['text'].tolist()

def boolean_search(query, inverted_index, documents):
    query_terms = [preprocess_text(term) for term in query.split()]
    result_docs = set(range(len(documents))) # Start with all documents

    for term in query_terms:
        if term.startswith("-"):  # NOT operator
            term = term[1:]
            try:
                result_docs -= set(inverted_index[term])
            except KeyError:
                pass  # Term not in index, ignore
        elif " or " in term.lower(): # OR operator
            sub_terms = [t.strip() for t in term.split(" or ")]
            or_docs = set()
            for sub_term in sub_terms:
                try:
                    or_docs.update(inverted_index[sub_term])
                except KeyError:
                    pass
            result_docs.intersection_update(or_docs)


        else:  # AND (default)
            try:
                result_docs.intersection_update(inverted_index[term])
            except KeyError:
                result_docs = set()  # Term not found, empty results
                break

    return [documents[doc_id] for doc_id in sorted(result_docs)]

def tfidf_search(query, documents):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    query_vec = vectorizer.transform([query])
    scores = (query_vec * tfidf_matrix.T).toarray()
    ranked_docs_indices = scores.argsort()[0][::-1]  # Sort descending
    return [documents[i] for i in ranked_docs_indices]

def bm25_search(query, documents):
    tokenized_corpus = [doc.split(" ") for doc in documents]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    ranked_docs_indices = doc_scores.argsort()[::-1]
    return [documents[i] for i in ranked_docs_indices]

def search_engine(query, inverted_index, documents, search_type="boolean"):

    if search_type == "tfidf":
        results = tfidf_search(query, documents)
    elif search_type == "bm25":
        results = bm25_search(query, documents)
    else:  # Default to boolean
        results = boolean_search(query, inverted_index, documents)

    return results

In [23]:
# Use our search engine
inverted_index_filepath = "/content/inverted_index.json"
documents_filepath = "/content/reduced_movie_plots.csv"

inverted_index = load_inverted_index(inverted_index_filepath)
documents = load_documents(documents_filepath)

query = input("Enter your search query: ")
search_type = input("Enter search type (boolean, tfidf, bm25): ") # User enters the query type to perform

results = search_engine(query, inverted_index, documents, search_type)

if results:
    print("\nSearch Results:")
    for i, result in enumerate(results[:20]): # Show first 20 documents
        print(f"{i+1}. {result[:150]}...") # Show first 150 characters of each result
        print("-" * 20)
else:
    print("No results found.")

Enter your search query: war OR prison
Enter search type (boolean, tfidf, bm25): boolean
No results found.


## Βήμα 5. Αξιολόγηση συστήματος

In [24]:
def preprocess_csv_with_doc_id(input_filepath, output_filepath="preprocessed_data_with_id.csv"):
    try:
        df = pd.read_csv(input_filepath, header=None, names=["text"])  # Or specify correct header if present
    except pd.errors.EmptyDataError:  # Handle empty CSV
        print(f"Error: CSV file '{input_filepath}' is empty or contains only headers.")
        return
    except FileNotFoundError:
        print(f"Error: File '{input_filepath}' not found.")
        return
    except pd.errors.ParserError as e:  # Handle parsing errors
        print(f"Error parsing CSV: {e}")
        return

    df['processed_text'] = df['text'].apply(preprocess_text)
    df['doc_id'] = range(len(df))
    df = df[['doc_id', 'processed_text']]
    df.to_csv(output_filepath, index=False, encoding="utf-8")
    print(f"Preprocessed data with doc_id saved to {output_filepath}")

def boolean_evaluation(queries, df):
    metrics = defaultdict(list)

    for query_text, keywords in queries.items():

        # Simulated relevance judgments:
        relevant_df = df[df['processed_text'].str.contains('|'.join(keywords), na=False, regex=True)]
        relevant_doc_ids = relevant_df['doc_id'].tolist()

        # Boolean search
        retrieved_df = df[df['processed_text'].str.contains('|'.join(keywords), na=False, regex=True)]
        retrieved_doc_ids = retrieved_df['doc_id'].tolist()


        all_doc_ids = set(df['doc_id'].tolist())
        relevant_docs_binary = [1 if doc_id in relevant_doc_ids else 0 for doc_id in all_doc_ids]
        retrieved_docs_binary = [1 if doc_id in retrieved_doc_ids else 0 for doc_id in all_doc_ids]



        if any(retrieved_docs_binary):  # Only calculate if documents were retrieved
            precision = precision_score(relevant_docs_binary, retrieved_docs_binary)
            recall = recall_score(relevant_docs_binary, retrieved_docs_binary)
            f1 = f1_score(relevant_docs_binary, retrieved_docs_binary)
        else:
            precision, recall, f1 = 0, 0, 0


        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["f1"].append(f1)

    return metrics

# Perform evaluation
input_filepath = "/content/reduced_movie_plots.csv"
preprocess_csv_with_doc_id(input_filepath)


df = pd.read_csv("preprocessed_data_with_id.csv")

queries = {
    "love": ["love"],
    "war": ["war"],
    "friend": ["friend"],
    "family": ["family"],
    "space": ["space"],
    "romantic comedy": ["romantic", "comedy"],
    "science fiction": ["science", "fiction"],
    "action thriller": ["action", "thriller"],
    "horror": ["horror"],
    "superhero": ["superhero"],
    "drama": ["drama"],
    "detective": ["detective"],
    "time travel": ["time", "travel"],
    "political thriller": ["political", "thriller"],
    "comedy": ["comedy"],
    "adventure": ["adventure"],
    "fantasy": ["fantasy"],
    "mystery": ["mystery"],
    "crime": ["crime"]
}

results = boolean_evaluation(queries, df)

for metric, values in results.items():
    mean_value = np.mean(values) if values else 0
    print(f"Mean {metric}: {mean_value:.4f}")

Preprocessed data with doc_id saved to preprocessed_data_with_id.csv
Mean precision: 0.5263
Mean recall: 0.5263
Mean f1: 0.5263
